# 1. Set up parameters for ASL

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from obspy import read_inventory
from importlib import reload
from flovopy.asl.wrappers2 import run_single_event, find_event_files, run_all_events
from flovopy.core.mvo import dome_location, REGION_DEFAULT
from flovopy.processing.sam import VSAM, DSAM 
from flovopy.asl.config import ASLConfig
# -------------------------- Config --------------------------
# directories
HOME = Path.home()
PROJECTDIR      = HOME / "Dropbox" / "BRIEFCASE" / "SSADenver"
LOCALPROJECTDIR = HOME / "work" / "PROJECTS" / "SSADenver_local"
OUTPUT_DIR      = LOCALPROJECTDIR / "asl_results"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
INPUT_DIR       = PROJECTDIR / "ASL_inputs" / "biggest_pdc_events"
GLOBAL_CACHE    = PROJECTDIR / "asl_global_cache"
METADATA_DIR    = PROJECTDIR / "metadata" 
STATION_CORRECTIONS_DIR = PROJECTDIR / "station_correction_analysis"

# master files
INVENTORY_XML   = METADATA_DIR / "MV_Seismic_and_GPS_stations.xml"
DEM_DEFAULT     = METADATA_DIR / "MONTSERRAT_DEM_WGS84_MASTER.tif"
GRIDFILE_DEFAULT= METADATA_DIR / "MASTER_GRID_MONTSERRAT.pkl"

# parameters for envelopes and cross-correlation
SMOOTH_SECONDS  = 1.0
MAX_LAG_SECONDS = 8.0
MIN_XCORR       = 0.5

# other parameters
DIST_MODE = "3d" # or 2d. will essentially squash Montserrat topography and stations onto a sea-level plane, ignored elevation data, e.g. for computing distances

# Inventory of Montserrat stations
from obspy import read_inventory
INV     = read_inventory(INVENTORY_XML)
print(f"[INV] Networks: {len(INV)}  Stations: {sum(len(n) for n in INV)}  Channels: {sum(len(sta) for net in INV for sta in net)}")

# Montserrat station corrections estimated from regionals
station_corrections_csv = STATION_CORRECTIONS_DIR / "station_gains_intervals.csv"
annual_station_corrections_csv = STATION_CORRECTIONS_DIR / "station_gains_intervals_by_year.csv"
station_corrections_df = pd.read_csv(station_corrections_csv)
annual_station_corrections_df = pd.read_csv(annual_station_corrections_csv)

# Montserrat pre-defined Grid (from 02 tutorial)
from flovopy.asl.grid import Grid
gridobj = Grid.load(GRIDFILE_DEFAULT)
print(gridobj)


# Montserrat constants
from flovopy.core.mvo import dome_location, REGION_DEFAULT
print("Dome (assumed source) =", dome_location)

# events and wrappers
from flovopy.asl.wrappers2 import run_single_event, find_event_files, run_all_events
event_files = list(find_event_files(INPUT_DIR))
eventcsvfile = Path(OUTPUT_DIR) / "mseed_files.csv"
if not eventcsvfile.is_file():
    rows = [{"num": num, "f": str(f)} for num, f in enumerate(event_files)]
    df = pd.DataFrame(rows)
    df.to_csv(eventcsvfile, index=False)
best_file_nums  = [35, 36, 40, 52, 82, 83, 84, 116, 310, 338]
best_event_files = [event_files[i] for i in best_file_nums]
print(f'Best miniseed files are: {best_event_files}')
REFINE_SECTOR = False   # enable triangular dome-to-sea refinement

# Parameters to pass for making pygmt topo maps
topo_kw = {
    "inv": INV,
    "add_labels": True,
    "cmap": "gray",
    "region": REGION_DEFAULT,
    "dem_tif": DEM_DEFAULT,  # basemap shading from your GeoTIFF - but does not actually seem to use this unless topo_color=True and cmap=None
    "frame": True,
    "dome_location": dome_location,
}

# Run events from last cell, one event at a time, to check it works

In [ ]:
# Build an ASL Configuration. This is inherited by various downstream functions
# This describes the physical parameters, the station metadata, the grid, the misfit algorithm, etc.
cfg = ASLConfig(
    inventory=INV,
    output_base=OUTPUT_DIR, # str?
    gridobj=gridobj,
    global_cache=GLOBAL_CACHE,
    station_correction_dataframe=station_corrections_df,
    wave_kind = "body", # "surface" or "body"
    speed = 3.0, # km/s
    Q = 100, # attenuation quality factor
    peakf = 2.0, # Hz
    dist_mode = DIST_MODE, # or "2d"
    misfit_engine = "r2", # l2, r2, lin?
    window_seconds = 5.0, # length of time window for amplitude measurement
    min_stations = 5, # minimum number of stations required to locate event
    sam_class = VSAM, # or DSAM
    sam_metric = "VT", # or one of "mean", "median", "max", "rms", "VLP", or "LP"
    debug=True,
)
cfg.build()
summaries = []

REFINE_SECTOR=False
for i, ev in zip(best_file_nums, best_event_files):
    print(f"[{i}/{len(event_files)}] {ev}")
    result = run_single_event(
        mseed_file=str(ev),
        cfg=cfg,
        refine_sector=REFINE_SECTOR,
        station_gains_df=None,
        switch_event_ctag = True,
        topo_kw=topo_kw,
        mseed_units='m/s', # default units for miniseed files being used - probably "Counts" or "m/s"        
        reduce_time=True,
        debug=True,
    )
    summaries.append(result)

# Summarize
df = pd.DataFrame(summaries)
display(df)

summary_csv = Path(OUTPUT_DIR) / f"{cfg.tag()}__summary.csv"
df.to_csv(summary_csv, index=False)
print(f"Summary saved to: {summary_csv}")

if not df.empty:
    n_ok = int((~df.get("error").notna()).sum()) if "error" in df.columns else len(df)
    print(f"Success: {n_ok}/{len(df)}")


In [ ]:
from dataclasses import replace
from pathlib import Path
import pandas as pd

# --- build the baseline cfg first (as you already do) ---
# cfg = ASLConfig(...).build()

def cfg_variants_from(baseline_cfg: ASLConfig):
    """
    Create ONE-change variants from a built baseline ASLConfig.
    Returns dict[label] -> built ASLConfig.
    """
    variants = {}

    # 1) Q 100 -> 23
    v_Q23 = replace(baseline_cfg, Q=23).build()
    variants["Q23"] = v_Q23

    # 2) speed 3.0 -> 1.5 km/s
    v_v15 = replace(baseline_cfg, speed=1.5).build()
    variants["v1.5"] = v_v15

    # 3) window 5 -> 2 s
    v_win2 = replace(baseline_cfg, window_seconds=2.0).build()
    variants["win2s"] = v_win2

    # 4) metric VT -> mean
    v_mean = replace(baseline_cfg, sam_metric="mean").build()
    variants["metric_mean"] = v_mean

    # 5) station corr ON -> OFF
    v_nosc = replace(baseline_cfg, station_correction_dataframe=None).build()
    variants["no_station_corr"] = v_nosc

    return variants


# --- make variants ---
variants = cfg_variants_from(baseline_cfg=cfg)   # dict[label] -> built ASLConfig

def csv_for_run(mseed_file: str | Path, cfg: ASLConfig) -> Path:
    """Locate the source CSV written by wrappers2.run_single_event for a given cfg."""
    mseed_file = Path(mseed_file)
    event_dir = Path(cfg.output_base) / mseed_file.stem
    products_dir = event_dir / Path(cfg.outdir).name
    refined = products_dir / f"{cfg.tag()}_refined.csv"
    plain   = products_dir / f"{cfg.tag()}.csv"
    return refined if refined.exists() else plain

# Build baseline config
cfg = ASLConfig(
    inventory=INV,
    output_base=OUTPUT_DIR,
    gridobj=gridobj,
    global_cache=GLOBAL_CACHE,
    station_correction_dataframe=station_corrections_df,
    wave_kind="body",
    speed=3.0,
    Q=100,
    peakf=2.0,
    dist_mode=DIST_MODE,
    misfit_engine="r2",
    window_seconds=5.0,
    min_stations=5,
    sam_class=VSAM,
    sam_metric="VT",
    debug=True,
).build()

# Generate variants (you’ll need cfg_variants_from() from wrappers2.py or config.py)
variants = cfg_variants_from(baseline_cfg=cfg)

REFINE_SECTOR = False
summaries = []

for i, ev in zip(best_file_nums, best_event_files):
    ev_key = Path(ev).stem
    print(f"\n[{i}/{len(event_files)}] Processing event: {ev_key}")

    for k, variant in variants.items():
        try:
            result = run_single_event(
                mseed_file=str(ev),
                cfg=variant,
                refine_sector=REFINE_SECTOR,
                station_gains_df=None,
                switch_event_ctag=True,
                topo_kw=topo_kw,
                mseed_units="m/s",
                reduce_time=True,
                debug=True,
            )
            summaries.append(result)
            print(f"  ✓ Variant {k} finished successfully")
        except Exception as e:
            print(f"  ✗ Variant {k} failed: {e}")



In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import math

# --- tiny geo helper ---
def _gc_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    p1 = math.radians(lat1); p2 = math.radians(lat2)
    dphi = p2 - p1
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dlmb/2)**2
    return 2*R*math.asin(min(1.0, math.sqrt(a)))

def _load_run_csv(path: str | Path):
    path = Path(path)
    if not path.exists():
        raise IOError(f"{path} does not exist")

    df = pd.read_csv(path)

    # case-insensitive column map
    cmap = {c.lower(): c for c in df.columns}
    def getcol(name, default=np.nan):
        key = name.lower()
        if key in cmap:
            return df[cmap[key]].to_numpy()
        return np.full(len(df), default)

    # parse time (t/time/utc/timestamp), normalize to UTC, round to seconds
    time_cols = [c for c in ("t", "time", "utc", "timestamp") if c in cmap]
    if time_cols:
        raw = df[cmap[time_cols[0]]]
        t_parsed = pd.to_datetime(raw, utc=True, errors="coerce")
        # count real timestamps
        n_real = t_parsed.notna().sum()
        if n_real > 0:
            t = t_parsed.dt.round("S").to_numpy("datetime64[ns]")
            time_kind = "real"
        else:
            t = np.arange(len(df))
            time_kind = "index"
    else:
        t = np.arange(len(df))
        time_kind = "index"

    out = {
        "t": t,
        "time_kind": time_kind,
        "lat": getcol("lat").astype(float),
        "lon": getcol("lon").astype(float),
        "DR": getcol("DR").astype(float) if "dr" in cmap else getcol("dr").astype(float),
        "misfit": getcol("misfit").astype(float),
        "azgap": getcol("azgap").astype(float),
        "connectedness": float(np.nanmean(getcol("connectedness"))) if "connectedness" in cmap else np.nan,
        "tag": path.stem,
    }
    return out

# --- aligner that falls back to index if time-intersection is empty ---
def _align_two(A, B, return_common: bool = False):
    """
    Align two runs:
      - If both have real timestamps, intersect on rounded-to-second time.
      - If no overlap on time (or one/both lack times), fall back to index alignment.
    Returns (A2, B2) or (A2, B2, common) if return_common=True.
    """
    def take(D, idx, common_vals):
        T = {}
        for k, v in D.items():
            if isinstance(v, np.ndarray) and v.shape[0] == D["t"].shape[0]:
                T[k] = v[idx]
            else:
                T[k] = v
        T["t"] = common_vals
        return T

    used_time = False
    if A["time_kind"] == "real" and B["time_kind"] == "real":
        # normalize/round to seconds for intersection
        tA = A["t"].astype("datetime64[s]").astype("datetime64[ns]")
        tB = B["t"].astype("datetime64[s]").astype("datetime64[ns]")
        common = np.intersect1d(tA, tB)
        if common.size > 0:
            idxA = {v: i for i, v in enumerate(tA)}
            idxB = {v: i for i, v in enumerate(tB)}
            iA = np.array([idxA[v] for v in common], dtype=int)
            iB = np.array([idxB[v] for v in common], dtype=int)
            used_time = True
        else:
            # fall back to index alignment
            n = min(A["t"].shape[0], B["t"].shape[0])
            if n <= 0:
                return None
            iA = np.arange(n, dtype=int)
            iB = np.arange(n, dtype=int)
            common = iA  # synthetic index
    else:
        # index alignment
        n = min(A["t"].shape[0], B["t"].shape[0])
        if n <= 0:
            return None
        iA = np.arange(n, dtype=int)
        iB = np.arange(n, dtype=int)
        common = iA

    A2 = take(A, iA, common)
    B2 = take(B, iB, common)
    if return_common:
        return A2, B2, common, ("time" if used_time else "index")
    return A2, B2

def compare_two_runs_csv(csvA, csvB, label="(baseline vs alt)"):
    if not csvA.is_file:
        print(f'{csvA} does not exist')
        return None
    if not csvB.is_file:
        print(f'{csvB} does not exist')
        return None
    print('both CSV files exist')
    def _debug_run_info(label, R):
        print(f"[{label}] kind={R['time_kind']}, n={R['t'].shape[0]}")
        if R["time_kind"] == "real" and R["t"].size:
            print("   first:", R["t"][0], " last:", R["t"][-1])

    # In compare_two_runs_csv just after loading:
    try:
        A = _load_run_csv(csvA); 
        B = _load_run_csv(csvB); 
    except:
        return None
    try:
        aligned = _align_two(A, B)
    except:
        _debug_run_info("A", A)
        _debug_run_info("B", B)
    if aligned is None:
        raise ValueError("No overlapping samples (time or index) between runs.")
    A, B = aligned

    latA, lonA = A["lat"], A["lon"]
    latB, lonB = B["lat"], B["lon"]

    # amplitude-based weights (mean DR of the two runs)
    DRw = np.nanmean(np.vstack([A["DR"], B["DR"]]), axis=0)
    DRw = np.where(np.isfinite(DRw), DRw, 0.0)
    w = DRw / (DRw.sum() + 1e-12) if DRw.max() > 0 else np.ones_like(DRw) / max(1, DRw.size)

    # per-sample spatial separation
    sep = np.array([
        _gc_km(latA[i], lonA[i], latB[i], lonB[i])
        if np.isfinite(latA[i]) and np.isfinite(lonA[i]) and np.isfinite(latB[i]) and np.isfinite(lonB[i])
        else np.nan
        for i in range(len(latA))
    ], dtype=float)

    mask = np.isfinite(sep)
    mean_sep   = float(np.nanmean(sep[mask])) if mask.any() else np.nan
    median_sep = float(np.nanmedian(sep[mask])) if mask.any() else np.nan
    wmean_sep  = float(np.nansum(sep * w)) if mask.any() else np.nan

    # simple % within thresholds
    pct_1km = float(100.0 * np.nanmean(sep <= 1.0)) if mask.any() else np.nan
    pct_2km = float(100.0 * np.nanmean(sep <= 2.0)) if mask.any() else np.nan
    pct_5km = float(100.0 * np.nanmean(sep <= 5.0)) if mask.any() else np.nan

    # (optional) path correlation on lat/lon by index overlap
    m = np.isfinite(latA) & np.isfinite(lonA) & np.isfinite(latB) & np.isfinite(lonB)
    lat_r = float(np.corrcoef(latA[m], latB[m])[0,1]) if np.count_nonzero(m) > 3 else np.nan
    lon_r = float(np.corrcoef(lonA[m], lonB[m])[0,1]) if np.count_nonzero(m) > 3 else np.nan

    # misfit / azgap averages and deltas
    mean_misfit_A = float(np.nanmean(A["misfit"]))
    mean_misfit_B = float(np.nanmean(B["misfit"]))
    d_misfit = mean_misfit_B - mean_misfit_A

    mean_azgap_A = float(np.nanmean(A["azgap"]))
    mean_azgap_B = float(np.nanmean(B["azgap"]))
    d_azgap = mean_azgap_B - mean_azgap_A

    return {
        "runA_tag": A["tag"],
        "runB_tag": B["tag"],
        "label": label,
        "align_mode": "time" if (A["time_kind"]=="real" and B["time_kind"]=="real") else "index",
        "n_overlap": int(A["t"].shape[0]),
        "mean_sep_km": mean_sep,
        "median_sep_km": median_sep,
        "amp_weighted_mean_sep_km": wmean_sep,
        "pct_within_1km": pct_1km,
        "pct_within_2km": pct_2km,
        "pct_within_5km": pct_5km,
        "lat_corr": lat_r,
        "lon_corr": lon_r,
        "mean_misfit_A": mean_misfit_A,
        "mean_misfit_B": mean_misfit_B,
        "delta_misfit_B_minus_A": d_misfit,
        "mean_azgap_A": mean_azgap_A,
        "mean_azgap_B": mean_azgap_B,
        "delta_azgap_B_minus_A": d_azgap,
        "connectedness_A": A["connectedness"],
        "connectedness_B": B["connectedness"],
        "delta_connectedness_B_minus_A": float(B["connectedness"] - A["connectedness"]) \
            if (np.isfinite(B["connectedness"]) and np.isfinite(A["connectedness"])) else np.nan,
    }



def safe_compare(summary_csv, csvA, csvB, label):
    csvA, csvB = Path(csvA), Path(csvB)
    if not csvA.exists():
        print(f"[skip] missing baseline: {csvA}")
        return None
    if not csvB.exists():
        print(f"[skip] missing variant:  {csvB}")
        return None
    try:
        row = compare_two_runs_csv(csvA, csvB, label)
    except Exception as e:
        print(f"[skip] compare failed ({label}): {e}")
        return None
    out = Path(summary_csv)
    df = pd.DataFrame([row])
    if out.exists():
        df0 = pd.read_csv(out)
        df = pd.concat([df0, df], ignore_index=True)
    df.to_csv(out, index=False)
    print(f"[compare] appended to {out} ({row['align_mode']} alignment, n={row['n_overlap']})")
    return row

for i, ev in zip(best_file_nums, best_event_files):
    print(f"[{i}/{len(event_files)}] {ev}")
    mseed_file=str(ev)

    # Build paths for baseline + variants for ONE event:
    event_dir    = OUTPUT_DIR / Path(mseed_file).stem
    products_dir = event_dir / Path(cfg.outdir).name

    baseline = products_dir / "source_VSAM_VT_5s_body_v3_Q100_F2_3d_r2_SC.csv"
    alt_Q23  = event_dir / "VSAM_VT_5s_body_v3_Q23_F2_3d_r2_SC" / "source_VSAM_VT_5s_body_v3_Q23_F2_3d_r2_SC.csv"
    alt_v15  = event_dir / "VSAM_VT_5s_body_v1.5_Q100_F2_3d_r2_SC" / "source_VSAM_VT_5s_body_v1.5_Q100_F2_3d_r2_SC.csv"
    alt_win2 = event_dir / "VSAM_VT_2s_body_v3_Q100_F2_3d_r2_SC" / "source_VSAM_VT_2s_body_v3_Q100_F2_3d_r2_SC.csv"
    alt_mean = event_dir / "VSAM_mean_5s_body_v3_Q100_F2_3d_r2_SC" / "source_VSAM_mean_5s_body_v3_Q100_F2_3d_r2_SC.csv"
    alt_nosc = event_dir / "VSAM_VT_5s_body_v3_Q100_F2_3d_r2"  / "source_VSAM_VT_5s_body_v3_Q100_F2_3d_r2.csv"# station corr OFF

    summary_csv = event_dir / "pairwise_run_comparisons.csv"

    # Append comparisons (CSV vs CSV; aligns by time if present, else by index)
    safe_compare(summary_csv, baseline, alt_Q23,  label="Q 100→23")
    safe_compare(summary_csv, baseline, alt_v15,  label="speed 3.0→1.5 km/s")
    #safe_compare(summary_csv, baseline, alt_win2, label="window 5→2 s")
    safe_compare(summary_csv, baseline, alt_mean, label="metric VT→mean")
    safe_compare(summary_csv, baseline, alt_nosc, label="station corr ON→OFF")

In [ ]:
# --- Run ASL per event (cell 6) ---
'''
from typing import List, Dict, Any
summaries: List[Dict[str, Any]] = []

for i, ev in zip(best_file_nums, best_event_files):
    print(f"[{i}/{len(event_files)}] {ev}")
    result = run_single_event(
        mseed_file=str(ev),
        cfg=cfg,
        refine_sector=REFINE_SECTOR,
        station_gains_df=None,
        topo_kw=topo_kw,
        debug=True,
    )
    summaries.append(result)
    break

# Summarize
df = pd.DataFrame(summaries)
display(df)

summary_csv = Path(OUTPUT_DIR) / f"{cfg.tag()}__summary.csv"
df.to_csv(summary_csv, index=False)
print(f"Summary saved to: {summary_csv}")

if not df.empty:
    n_ok = int((~df.get("error").notna()).sum()) if "error" in df.columns else len(df)
    print(f"Success: {n_ok}/{len(df)}")
'''

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

def load_all_event_comparisons(root: Path) -> pd.DataFrame:
    """
    Crawl event folders under `root` and stack `pairwise_run_comparisons.csv`.
    Returns a tidy DF with event_id inferred from folder name.
    """
    rows = []
    for csv in root.rglob("pairwise_run_comparisons.csv"):
        try:
            df = pd.read_csv(csv)
            df["event_id"] = csv.parent.name            # the event folder name
            rows.append(df)
        except Exception as e:
            print(f"[skip] {csv}: {e}")
    if not rows:
        return pd.DataFrame()
    out = pd.concat(rows, ignore_index=True)
    # normalize label text to a short key
    out["variant"] = out["label"].astype(str)
    # guard presence of expected columns
    for c in ["mean_sep_km","delta_misfit_B_minus_A","delta_azgap_B_minus_A"]:
        if c not in out.columns: out[c] = np.nan
    return out

def add_composite_score(df: pd.DataFrame,
                        w_sep=1.0, w_misfit=0.5, w_azgap=0.1) -> pd.DataFrame:
    """
    Lower is better. Negative deltas are good if they reduce misfit/azgap.
    """
    d = df.copy()
    # z-score each metric for comparability (event-wise optional)
    # here: global z-scores; switch to per-event z if events differ strongly in scale
    for col in ["mean_sep_km","delta_misfit_B_minus_A","delta_azgap_B_minus_A"]:
        x = d[col].to_numpy(dtype=float)
        mu, sd = np.nanmean(x), np.nanstd(x) if np.nanstd(x)>0 else 1.0
        d[col+"_z"] = (x - mu)/sd
    d["score"] = (
        w_sep    * d["mean_sep_km_z"] +
        w_misfit * d["delta_misfit_B_minus_A_z"] +
        w_azgap  * d["delta_azgap_B_minus_A_z"]
    )
    return d

def summarize_variants(df: pd.DataFrame) -> pd.DataFrame:
    """
    One line per variant: mean±SE of core metrics and composite score,
    plus 'wins' (how often variant beats baseline the most for an event).
    """
    g = df.groupby("variant", dropna=False)
    agg = g.agg(
        n_events          = ("event_id", "nunique"),
        n_rows            = ("event_id", "size"),
        mean_sep_km_mean  = ("mean_sep_km", "mean"),
        mean_sep_km_med   = ("mean_sep_km", "median"),
        mean_sep_km_se    = ("mean_sep_km", lambda x: np.nanstd(x)/np.sqrt(max(1,(x.notna().sum())))),
        dmisfit_mean      = ("delta_misfit_B_minus_A", "mean"),
        dmisfit_med       = ("delta_misfit_B_minus_A", "median"),
        dazgap_mean       = ("delta_azgap_B_minus_A", "mean"),
        score_mean        = ("score", "mean"),
        score_med         = ("score", "median"),
    ).reset_index().sort_values("score_mean")
    return agg

def per_event_winner(df_scored: pd.DataFrame) -> pd.DataFrame:
    """
    For each event, pick the variant with the lowest composite score.
    """
    # keep only the best per (event_id)
    idx = df_scored.groupby("event_id")["score"].idxmin()
    winners = df_scored.loc[idx, ["event_id","variant","score"]]
    win_counts = winners.groupby("variant").size().rename("wins").reset_index()
    return winners, win_counts.sort_values("wins", ascending=False)

# --- run it ---
ROOT = OUTPUT_DIR  # your existing OUTDIR base
allcmp = load_all_event_comparisons(ROOT)
print(f"stacked rows: {len(allcmp)}, events: {allcmp['event_id'].nunique()}")

scored = add_composite_score(allcmp, w_sep=1.0, w_misfit=0.5, w_azgap=0.1)
summary = summarize_variants(scored)
winners, win_counts = per_event_winner(scored)

# quick looks
display(summary.head(10))
display(win_counts)

# Run all events efficiently

In [ ]:
print(INPUT_DIR)
print(cfg)
print(topo_kw)
print(REFINE_SECTOR)
'''
run_all_events(
    input_dir=INPUT_DIR,
    station_gains_df = None,
    cfg=cfg,
    refine_sector=REFINE_SECTOR,
    topo_kw=topo_kw,
    debug=True,
    max_events=999999,
    use_multiprocessing=True,
    workers=4,
)
'''

# Run Monte Carlo sweep of parameters for 1 event


In [ ]:

from flovopy.asl.wrappers2 import run_event_monte_carlo
from flovopy.processing.sam import VSAM, DSAM
'''
# Simple 6-draw sweep (replace with your own priors/sequences)
configs = ASLConfig.generate_config_list(
    inventory=None,
    output_base=None,
    gridobj=None,
    global_cache=None,      
    wave_kinds=("surface","body"),
    station_corr_tables=(station_corrections_df), #annual_station_corrections_df),
    speeds=(1.0, 3.0),
    Qs=(23, 1000),
    dist_modes=("3d",), # 2d needs a different grid and different distance and amplitude corrections
    misfit_engines=("l2","r2", "lin"),
    peakfs=(2.0, 8.0),
    window_seconds = 5.0, # change to be a tuple 10.0) not implemented yet
    min_stations = 5,
    sam_class = (VSAM), #, DSAM), # not implemented yet
    sam_metric = ("mean"),# "median", "rms", "VT", "LP"), # this doesn't seem to be implemented yet
    # context can be set later; set here if you like:
    debug=False,
)

configs = ASLConfig.generate_config_list(
    inventory=None,
    output_base=None,
    gridobj=None,
    global_cache=None,      
    wave_kinds=("surface",),
    station_corr_tables=(station_corrections_df), #annual_station_corrections_df),
    speeds=(1.0, 3.0),
    Qs=(23, 1000),
    dist_modes=("3d",), # 2d needs a different grid and different distance and amplitude corrections
    misfit_engines=("l2"),
    peakfs=(8.0),
    window_seconds = 5.0, # change to be a tuple 10.0) not implemented yet
    min_stations = 5,
    sam_class = (VSAM), #, DSAM), # not implemented yet
    sam_metric = ("mean"),# "median", "rms", "VT", "LP"), # this doesn't seem to be implemented yet
    # context can be set later; set here if you like:
    debug=False,
)


configs = ASLConfig.generate_config_list(    
    inventory=INV,
    output_base=str(OUTPUT_DIR),
    gridobj=gridobj,
    global_cache=GLOBAL_CACHE,
) 

print(len(configs))
'''


In [ ]:

# Shared run context
mseed_file   = event_files[116]
'''
results = run_event_monte_carlo(
    mseed_file=mseed_file,
    configs=configs,
    inventory=INV,
    output_base=str(OUTPUT_DIR),
    gridobj=gridobj,
    topo_kw=topo_kw,
    station_gains_df=None,
    parallel=False,
    max_workers=1,
    global_cache=GLOBAL_CACHE,
    debug=True,
)

# Inspect or summarize results as needed
n_ok = sum(1 for r in results if "error" not in r)
print(f"[MC] Completed {n_ok}/{len(results)} runs OK")
'''